In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Emachines\\Documents\\GitHub\\Prediction-of-Mohs-Hardness\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Emachines\\Documents\\GitHub\\Prediction-of-Mohs-Hardness'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [8]:
from mlProject import logger

from mlProject.components.data_cleaning import (FeatureEngineering,
                                                FixOutliers,
                                                DataPreprocessing)

In [9]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def data_transformation_pipeline(self):
        """Create a pipeline for data transformation

        Args:
            config (DataTransformationConfig): Data transformation configuration
        Returns:
            pipeline: Pipeline for data transformation
        """
        
        try:
            pipeline = Pipeline([
                ("Feature Engineering", FeatureEngineering()),
                ("Fix Outliers", FixOutliers()),
                ("Data Preprocessing", DataPreprocessing()),
            ])

            return pipeline
        except Exception as e:
            logger.error(f"Data transformation failed with the following error: {e}")
            raise e

    def transform_split_data(self):
        """Transform and split data into train and test sets
        
        Args:
            config (DataTransformationConfig): Data transformation configuration
        Returns:
            None
        """

        data = pd.read_csv(self.config.data_path)
        pipeline = self.data_transformation_pipeline()

        processed_data = pipeline.fit_transform(data)
        
        # Split data into train and test sets
        train, test = train_test_split(processed_data, test_size=0.2, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Pipeline transformation and data splitting completed successfully")

        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

        


In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_split_data()
except Exception as e:
    logger.error(f"Failed to transform and split data with the following error: {e}")
    raise e

[2023-12-01 01:01:12,800: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-01 01:01:12,805: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-01 01:01:12,815: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-01 01:01:12,841: INFO: common: created directory at: artifacts]
[2023-12-01 01:01:12,847: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-01 01:01:12,969: INFO: data_cleaning: Feature engineering completed successfully]
[2023-12-01 01:01:13,003: INFO: data_cleaning: allelectrons_Total outliers are values < -121.0 or > 320.0]
[2023-12-01 01:01:13,034: INFO: data_cleaning: atomicweight_ionenergy_Ratio outliers are values < -6.2820975950694535 or > 12.620787022963274]
[2023-12-01 01:01:13,061: INFO: data_cleaning: normalized_density outliers are values < -0.1617694136215771 or > 0.3980561817681934]
[2023-12-01 01:01:13,096: INFO: data_cleaning: el_neg_chi_R_vdw_Ratio outliers are values < 0.6513176